In [44]:
import pandas as pd
import sqlite3
import seaborn as sns   
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler



In [41]:
from data_loader import load_fire_data

df_functon = load_fire_data('../data/FPA_FOD_20170508.sqlite')

In [43]:
df_functon.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [3]:
# Path to the SQLite database file
db_path = '../data/FPA_FOD_20170508.sqlite'

# Connect to the database
conn = sqlite3.connect(db_path)

# List all tables in the database
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)
print("Tables in the database:")
print(tables)

Tables in the database:
                                  name
0                      spatial_ref_sys
1                   spatialite_history
2                      sqlite_sequence
3                     geometry_columns
4                  spatial_ref_sys_aux
5               views_geometry_columns
6               virts_geometry_columns
7          geometry_columns_statistics
8    views_geometry_columns_statistics
9    virts_geometry_columns_statistics
10        geometry_columns_field_infos
11  views_geometry_columns_field_infos
12  virts_geometry_columns_field_infos
13               geometry_columns_time
14               geometry_columns_auth
15         views_geometry_columns_auth
16         virts_geometry_columns_auth
17                  sql_statements_log
18                        SpatialIndex
19                ElementaryGeometries
20                                 KNN
21                               Fires
22                     idx_Fires_Shape
23                idx_Fires_Shape_node
2

In [4]:
# Load the 'Fires' table into a DataFrame
fires_df = pd.read_sql('SELECT * FROM Fires', conn)

# Preview the first 5 records
print(fires_df.head())

   OBJECTID  FOD_ID      FPA_ID SOURCE_SYSTEM_TYPE SOURCE_SYSTEM  \
0         1       1  FS-1418826                FED   FS-FIRESTAT   
1         2       2  FS-1418827                FED   FS-FIRESTAT   
2         3       3  FS-1418835                FED   FS-FIRESTAT   
3         4       4  FS-1418845                FED   FS-FIRESTAT   
4         5       5  FS-1418847                FED   FS-FIRESTAT   

  NWCG_REPORTING_AGENCY NWCG_REPORTING_UNIT_ID  NWCG_REPORTING_UNIT_NAME  \
0                    FS                USCAPNF    Plumas National Forest   
1                    FS                USCAENF  Eldorado National Forest   
2                    FS                USCAENF  Eldorado National Forest   
3                    FS                USCAENF  Eldorado National Forest   
4                    FS                USCAENF  Eldorado National Forest   

  SOURCE_REPORTING_UNIT SOURCE_REPORTING_UNIT_NAME  ... FIRE_SIZE_CLASS  \
0                  0511     Plumas National Forest  ...    

In [6]:
# Check nulls
print(fires_df.isnull().sum())

OBJECTID                            0
FOD_ID                              0
FPA_ID                              0
SOURCE_SYSTEM_TYPE                  0
SOURCE_SYSTEM                       0
NWCG_REPORTING_AGENCY               0
NWCG_REPORTING_UNIT_ID              0
NWCG_REPORTING_UNIT_NAME            0
SOURCE_REPORTING_UNIT               0
SOURCE_REPORTING_UNIT_NAME          0
LOCAL_FIRE_REPORT_ID          1459286
LOCAL_INCIDENT_ID              820821
FIRE_CODE                     1555636
FIRE_NAME                      957189
ICS_209_INCIDENT_NUMBER       1854748
ICS_209_NAME                  1854748
MTBS_ID                       1869462
MTBS_FIRE_NAME                1869462
COMPLEX_NAME                  1875282
FIRE_YEAR                           0
DISCOVERY_DATE                      0
DISCOVERY_DOY                       0
DISCOVERY_TIME                 882638
STAT_CAUSE_CODE                     0
STAT_CAUSE_DESCR                    0
CONT_DATE                      891531
CONT_DOY    

In [27]:
# Drop columns with excessive missing values
drop_cols = [
    'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
    'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
    'COMPLEX_NAME', 'FIPS_CODE', 'FIPS_NAME']

df = fires_df.drop(columns=drop_cols)
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,1730,0.10,A,40.036944,-121.005833,5.0,USFS,CA,63,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,1530,0.25,A,38.933056,-120.404444,5.0,USFS,CA,61,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,2024,0.10,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,1400,0.10,A,38.559167,-119.913333,5.0,USFS,CA,3,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,1200,0.10,A,38.559167,-119.933056,5.0,USFS,CA,3,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [28]:
print(df.columns.tolist())

['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'Shape']


In [29]:
time_columns = ['DISCOVERY_TIME', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME']

# Check which of these columns actually exist
existing_time_columns = [col for col in time_columns if col in df.columns]
print("Existing time columns:", existing_time_columns)

df = df.dropna(subset=existing_time_columns)

print(f"Remaining rows after dropping time-related NaNs: {len(df)}")


Existing time columns: ['DISCOVERY_TIME', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME']
Remaining rows after dropping time-related NaNs: 892007


In [30]:
def time_to_hour(time_str):
    if pd.isna(time_str):
        return np.nan
    time_str = str(int(time_str)).zfill(4)
    return int(time_str[:2])

df['DISCOVERY_HOUR'] = df['DISCOVERY_TIME'].apply(time_to_hour)
df['CONT_HOUR'] = df['CONT_TIME'].apply(time_to_hour)


In [31]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'], unit='D', origin='julian')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'], unit='D', origin='julian')

df['FIRE_DURATION'] = (df['CONT_DATE'] - df['DISCOVERY_DATE']).dt.days
df['FIRE_DURATION'] = df['FIRE_DURATION'].fillna(-1)  # -1 for unknown duration


In [32]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,Shape,DISCOVERY_HOUR,CONT_HOUR,FIRE_DURATION
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,40.036944,-121.005833,5.0,USFS,CA,63,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,13,17,0
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,38.933056,-120.404444,5.0,USFS,CA,61,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,8,15,0
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,19,20,0
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,38.559167,-119.913333,5.0,USFS,CA,3,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,16,14,5
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,38.559167,-119.933056,5.0,USFS,CA,3,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,16,12,5


In [33]:
median_discovery_hour = df['DISCOVERY_HOUR'].median()
median_cont_hour = df['CONT_HOUR'].median()

df['DISCOVERY_HOUR'].fillna(median_discovery_hour, inplace=True)
df['CONT_HOUR'].fillna(median_cont_hour, inplace=True)


/var/folders/b5/c2s8x6hx5bz57tz7p2pf4f2h0000gn/T/ipykernel_46607/1190890236.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DISCOVERY_HOUR'].fillna(median_discovery_hour, inplace=True)
/var/folders/b5/c2s8x6hx5bz57tz7p2pf4f2h0000gn/T/ipykernel_46607/1190890236.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [34]:
label_cols = ['STATE', 'STAT_CAUSE_DESCR', 'OWNER_DESCR']
le = LabelEncoder()

for col in label_cols:
    df[col] = le.fit_transform(df[col])

In [35]:

# Season based on DISCOVERY_DATE
def get_season(date):
    if pd.isnull(date):
        return 'Unknown'
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['SEASON'] = df['DISCOVERY_DATE'].apply(get_season)


# Simplify causes
human_causes = ['Arson', 'Debris Burning', 'Equipment Use', 'Smoking', 'Campfire', 'Children', 'Fireworks']
natural_causes = ['Lightning']

def simplify_cause(cause):
    if cause in human_causes:
        return 'Human'
    elif cause in natural_causes:
        return 'Natural'
    else:
        return 'Unknown'

df['CAUSE_SIMPLE'] = df['STAT_CAUSE_DESCR'].apply(simplify_cause)


In [36]:
df['DISCOVERY_HOUR_MISSING'] = df['DISCOVERY_HOUR'].isna().astype(int)
df['CONT_HOUR_MISSING'] = df['CONT_HOUR'].isna().astype(int)


In [37]:
print(df.columns.to_list())

['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'Shape', 'DISCOVERY_HOUR', 'CONT_HOUR', 'FIRE_DURATION', 'SEASON', 'CAUSE_SIMPLE', 'DISCOVERY_HOUR_MISSING', 'CONT_HOUR_MISSING']


In [38]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

Missing values per column:
OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
NWCG_REPORTING_UNIT_NAME           0
SOURCE_REPORTING_UNIT              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_DOY                      0
DISCOVERY_TIME                     0
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
CONT_DATE                          0
CONT_DOY                           0
CONT_TIME                          0
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_CODE                         0
OWNER_DESCR                        0
STATE                              0
COUNTY     

In [42]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,STATE,COUNTY,Shape,DISCOVERY_HOUR,CONT_HOUR,FIRE_DURATION,SEASON,CAUSE_SIMPLE,DISCOVERY_HOUR_MISSING,CONT_HOUR_MISSING
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,4,63,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,13,17,0,Winter,Unknown,0,0
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,4,61,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,8,15,0,Spring,Unknown,0,0
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,4,17,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,19,20,0,Spring,Unknown,0,0
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,4,3,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,16,14,5,Summer,Unknown,0,0
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,4,3,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,16,12,5,Summer,Unknown,0,0


In [46]:
from sklearn.preprocessing import StandardScaler

# Columns to scale
scale_cols = [
    'LATITUDE', 'LONGITUDE', 'DISCOVERY_DOY', 'DISCOVERY_HOUR',
    'CONT_DOY', 'CONT_HOUR', 'FIRE_DURATION',
]

scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

print(df[scale_cols].head())


   LATITUDE  LONGITUDE  DISCOVERY_DOY  DISCOVERY_HOUR  CONT_DOY  CONT_HOUR  \
0  0.285988  -1.354134      -1.670664       -0.313641 -1.672906   0.417868   
1  0.093052  -1.317076      -0.471006       -1.547012 -0.481295  -0.043689   
2  0.101985  -1.337479      -0.243071        1.166405 -0.254889   1.110203   
3  0.027704  -1.286813       0.092833        0.426382  0.138343  -0.274468   
4  0.027704  -1.288029       0.092833        0.426382  0.138343  -0.736025   

   FIRE_DURATION  
0      -0.090048  
1      -0.090048  
2      -0.090048  
3       0.271755  
4       0.271755  
